In [1]:
from transformers import AutoTokenizer
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
import os


e:\Laby\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
all_pages = []
folder_path = 'documents'
for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path,file)
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    all_pages.extend(pages)
print(len(all_pages))

84


In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)
docs = text_splitter.split_documents(pages)

In [7]:
model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1" 
embeddings = HuggingFaceEmbeddings(model_name=model_name)

C:\Users\nukle\AppData\Local\Temp\ipykernel_22028\682296968.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [8]:
db = FAISS.from_documents(docs, embeddings)
db.save_local("faiss_index_hf_custom")

In [9]:
model_name = "google/flan-t5-base" # użycie innego modelu
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [10]:
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024
)

Device set to use cpu


In [11]:
llm = HuggingFacePipeline(pipeline=pipe)

C:\Users\nukle\AppData\Local\Temp\ipykernel_22028\2691217058.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [12]:
memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True,
    output_key = "answer"
)

C:\Users\nukle\AppData\Local\Temp\ipykernel_22028\1053975571.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [13]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 3}),  # pobiera top-3 fragmenty
    memory=memory,
    return_source_documents=True,
    output_key = "answer"
)

In [14]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(),
    memory=memory,
    return_source_documents=True,
    output_key="answer"
)

In [18]:
def print_history(memory):
    history = memory.load_memory_variables({})["chat_history"]
    for message in history:
        who = "User" if message.type == "human" else "AI"
        print(f"\n{who}: {message.content}")

In [20]:
def ask_question(query):
    result = qa_chain.invoke({"question": query},return_only_outputs=True)
    
    if not result["source_documents"]:
        return "Information not found in documents"
    
    answer = result["answer"]
    sources = []
    for doc in result["source_documents"]:
        file = doc.metadata.get("source", "uknown source")
        page = doc.metadata.get("page", "brak")
        sources.append(f"{os.path.basename(file)} (s. {page+1})")  
    
    sources_str = ", ".join(set(sources))  
    
    return f"{answer}\n\Source: {sources_str}"


<>:16: SyntaxWarning: invalid escape sequence '\S'
<>:16: SyntaxWarning: invalid escape sequence '\S'
C:\Users\nukle\AppData\Local\Temp\ipykernel_22028\1629347746.py:16: SyntaxWarning: invalid escape sequence '\S'
  return f"{answer}\n\Source: {sources_str}"


In [19]:
def start_interace():
    print("Interface ready \n")
    print("Commands:")
    print("exit : quit the interface")
    print("history : print chat history \n")
    
    while True:
        user_q = input("Ty: ").strip()
        if user_q == "exit":
            print("Interface succesfully closed")
            break
        if user_q == "history":
            print_history(memory)
            continue

        try:
            answer = ask_question(user_q)
            print(f"\n User: {user_q}")
            print(f"\n AI: {answer} \n")
        except KeyboardInterrupt:
            print("interrupted")
            break

In [21]:
start_interace()

Interface ready 

Commands:
exit : quit the interface
history : print chat history 


User: What is attention?

AI: information interactions inside trans- former

User: How NLP models detect AI writing in education?

AI: by using structural probing [64], structured perceptron parsers [65]) or visualization (e.g., as and Short Papers

User: What are encoders and decoders?

AI: a set of multi-head attention layers, add and normalize and feed- forward layers. The multi-head attention mechanism provides the model with the ability to orchestrate information from dif- Its outputs are fed either in other encoders or into decoders, depending on the architecture. There is no fixed number of encoders and decoders which can be included in this architecture, but they will typically be paired (e.g., 10 en- coders and 10 decoders). In newer architectures, encoders and the model with the ability to orchestrate information from dif- ferent representation subspaces (e.g., multiple weight matrices) at v

**Odpowiedź na pytania**

Interface ready 

Commands:
exit : quit the interface
history : print chat history 


 User: What is attention?

 AI: information interactions inside trans- former
\Source: Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 7), Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 8), Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 9) 


 User: How NLP models detect AI writing in education?

 AI: by using structural probing [64], structured perceptron parsers [65]) or visualization (e.g., as and Short Papers
\Source: Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 3), Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 1), Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 9), Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 4) 


 User: What are encoders and decoders?

 AI: a set of multi-head attention layers, add and normalize and feed- forward layers. The multi-head attention mechanism provides the model with the ability to orchestrate information from dif- Its outputs are fed either in other encoders or into decoders, depending on the architecture. There is no fixed number of encoders and decoders which can be included in this architecture, but they will typically be paired (e.g., 10 en- coders and 10 decoders). In newer architectures, encoders and the model with the ability to orchestrate information from dif- ferent representation subspaces (e.g., multiple weight matrices) at various positions (e.g., different words in a sentence) [1].
\Source: Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 1) 


 User: What is transformer?

 AI: The Transformer model itself is simple and consists from an encoder and a decoder. The encoder typically contains a set with Transformers, we typically need to understand all the facets of the problem, from the data and training procedure, to the input, network layers, weights or various outputs of the neural network. This can sometimes be accomplished by using model-agnostic tools specifically built for benchmarking
\Source: Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 6), Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 2), Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 1), Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 4) 


 User: What is the role of Long-Range Transformers?

 AI: We typically need to understand all the facets of the problem, from the data and training procedure, to the input, network layers, weights or various outputs of the neural network. This can sometimes be accomplished by using model-agnostic tools specifically built for benchmarking that can be used to explain BERT predictions; and (ii) custom visualizations that are focused only on explaining the inner workings of Transformers.
\Source: Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 2), Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 1) 


 User: What is quantum mechanic?

 AI: A tool for visual analysis of hidden state dynamics
\Source: Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 8), Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 7) 

User: How long should wine ferment?

 AI: a few days
\Source: Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 8), Visualizing_Transformers_for_NLP_A_Brief_Survey.pdf (s. 9) 

Interface succesfully closed

**Podsumowanie**

Model w miarę sensownie odpowiada na pytania, nawet takie, które nie nawiązują do dokumentów źródłowych, np. odpowiedź na pytanie "How long should wine ferment?", "What is quantum mechanic?".  
Jednak jest to halucynacja. 

Niestety brakuje tutaj leszego mechanizmu zapobiegającemu halucynacji oraz dokładniejszego cytowania. 

